In [5]:
# Save Model Using joblib
import psycopg2
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib


In [6]:
conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

In [7]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [8]:
cursor.execute('Select * from "ahshouseholdclass2"')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

df = pd.DataFrame(data=rows, columns=col_names )

In [9]:
LABEL_MAP = {
"(90000.0, 100000.0]": 10,
"(80000.0, 90000.0]": 9,
"(70000.0, 80000.0]": 8,   
"(60000.0, 70000.0]": 7,
"(50000.0, 60000.0]": 6,
"(40000.0, 50000.0]": 5,
"(30000.0, 40000.0]": 4,
"(20000.0, 30000.0]": 3,
"(10000.0, 20000.0]": 2,
"(0.0, 10000.0]": 1}

# Convert categorical labels into incremental value
df['HINCP_BIN'] = df['HINCP_BIN'].map(LABEL_MAP).to_frame()

### Class Balanced

In [10]:
# Create the train and test data
from sklearn.model_selection import train_test_split

X = df[['HHAGE', 'HINCP_BIN', 'UTILAMT','UNITSIZE','NUMPEOPLE','BATHROOMS', 
        'PERPOVLVL',  'ELECAMT', 'GASAMT', 
        'TRASHAMT', 'WATERAMT', 'OMB13CBSA','STORIES', 'HHNATVTY'
       ]]

y = df['RATINGHS_BIN']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

In [11]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 33)

In [12]:
X_sm, y_sm = sm.fit_sample(X, y)


In [13]:
X_sm, y_sm = sm.fit_sample(X_sm, y_sm.ravel())
pd.Series(y_sm).value_counts().plot.bar()

In [15]:
# Fit the model on training set
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
# save the model to disk
filename = 'test_model.sav'
joblib.dump(model, filename)

['test_model.sav']